**Instalación de librerías necesarias**

In [43]:
!pip install librosa==0.9.2 
!pip install numpy
!pip install IPython
!pip install --upgrade setuptools
!pip install pandas
!pip install matplotlib
!pip install rtree
!pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


**Importación de librerías**:

In [8]:
import librosa
import IPython.display as ipd
import os
import numpy as np
import pandas as pd
import heapq
import time
import matplotlib.pyplot as plt
import concurrent.futures
import rtree
import faiss

**Definición de funciones**:

In [9]:
# Funciones de extraccion de caracteristicas
def get_audios(n):
    audios = []
    for i in range(1, 156):
        num = ""
        if i < 10:
            num = "00" + str(i)
        elif i < 100:
            num = "0" + str(i)
        else:
            num = str(i)

        for file in os.listdir('fma_medium/' + num):
            if file.endswith('.mp3'):
                audios.append('fma_medium/' + num + '/' + file)
            if len(audios) == n:
                return audios

def features_extraction(file_path, dimensions):
    #load the audio file
    x, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    #extract features from the audio file
    mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=dimensions).T, axis=0) # axis = 1
    
    return mfcc

def extract_n_features(n, dimensions, threads):
    audios = get_audios(n)
    features = []
    results = {}

    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        future_to_data = {executor.submit(features_extraction, item, dimensions): item for item in audios }
        for future in concurrent.futures.as_completed(future_to_data):
            data_item = future_to_data[future]
            try:
                result = future.result()
                # Assign the result to the input string in the dictionary
                results[data_item] = result
            except Exception as exc:
                print(f'{data_item} generated an exception: {exc}')
    return results

# Funciones de KNN Range priority queue
def euclidean_distances(query, data):
    return np.sqrt(((data - query) ** 2).sum(axis=1))

def KNNPriorityQueue(query, K, chars):
    data = np.loadtxt(chars, delimiter=',')
    
    if len(data) < K:
        raise ValueError("El número de datos es menor que K")
    
    distances = euclidean_distances(query, data)
    pq = []
    
    # Insertamos los primeros K elementos de la base de datos en la cola de prioridad
    for i in range(K):
        heapq.heappush(pq, (-distances[i], i))  # Distancia negativa para simular una cola de prioridad máxima

    for i in range(K, len(data)):
        # Si la distancia entre el objeto de consulta y el objeto i es menor que la mayor distancia en la cola de prioridad
        if -pq[0][0] > distances[i]: # Eliminamos el objeto en la cola de prioridad con mayor distancia
            heapq.heappop(pq)
            heapq.heappush(pq, (-distances[i], i))
            
    return [(data[i], -distance) for distance, i in sorted(pq, reverse=True)]

def knnSearch(collection, query, k):
    heap = [] # heap vacio
    ED = lambda P, Q: np.sqrt((P-Q)**2)

    for i in range(len(collection)):
        dist = np.sqrt((ED(collection[i], query) ** 2).sum()) # porque es una serie de pandas y no un array de numpy
        # Use negative distance because heapq is a min heap
        if len(heap) < k:
            heapq.heappush(heap, (-dist, i)) # si no se ha llenado el heap, se inserta
        else:
            heapq.heappushpop(heap, (-dist, i)) # si ya se lleno el heap, se inserta y se elimina el menor

    # Return indices and distances, reversing the order so that the closest is first
    indices_and_distances = [(i, -d) for d, i in sorted(heap, reverse=True)]
    return indices_and_distances
 
# Funciones para KNN Range radio
def genDistancias(data, N):
    ED = lambda P, Q: np.sqrt(sum((P-Q)**2))
    v = np.zeros(N)
    for i in range(N):
        ind = np.random.choice(data.shape[0], size=2, replace=False)
        P = data[ind[0], :]
        Q = data[ind[1], :]
        #v[i] = distance.euclidean(P, Q)
        v[i] = np.sqrt((ED(P, Q) ** 2).sum())
    return v

def rangeSearch(collection, query, r, query_index):
    ED = lambda P, Q: np.sqrt(sum((P-Q)**2))
    result = []
    heap = []
    for i in range(len(collection)):
        dist = np.sqrt((ED(collection[i], query) ** 2).sum())
        if (dist < r).all():  # Check if all elements in the Series are less than r
            result.append(i)
            #sort by distance to r
            heapq.heappush(heap, (dist, i))
    sorted_result = [index for _, index in heapq.nsmallest(len(heap), heap)]
    return sorted_result # result

# Funciones para KNN Rtree
def knn_rtree(collection, query, k, dimensions):
    # Create a new RTree index
    prop = rtree.index.Property()
    prop.dimension = dimensions   # dimension del vector caracteristico
    #prop.buffering_capacity = 8    # Cantidad maxima de MBRs en un nodo 
    ind = rtree.index.Index(properties = prop)

    # insertar los puntos                
    for i in range(collection.shape[0]):
        ind.insert(i, collection[i].tolist() + collection[i].tolist())     

    # Promedio
    avg = 0
    # aplicar la consulta indexada (correr 10 veces y tomar el promedio)
    for i in range(10):
        start_time = time.time()
        ind.nearest(query, num_results=k) # cambiar a k+1 porque se cuenta a si mismo
        avg += time.time() - start_time

    avg = avg/10
    # Obtener los k vecinos más cercanos (k+1 para excluir el punto de consulta si está presente)
    k_nearest = list(ind.nearest(query.tolist() + query.tolist(), num_results=k))
    ind.close()
    
    neighbors = collection[k_nearest]

    # Calcular distancias y ordenar
    distances = []
    for idx in k_nearest:
        dist = np.linalg.norm(collection[idx] - query)
        distances.append((dist, idx))
            
    distances.sort(key=lambda x: x[0])  # Ordenar por distancia
    
    # Extraer índices ordenados
    sorted_indices = [idx for _, idx in distances]

    print(f'La consulta tomó en promedio {avg} segundos')
    #print(f'Los k={k} más cercanos a la consulta son:{neighbors_without_query}')
    print(f'Las k:{k} canciones más cercanas a la consulta son (contar la query): {sorted_indices}')
    #print(f'Las k:{k} canciones más cercanas a la consulta son (sin contar la query): {nearest_songs}')

# Funciones para HighD
def knn_faiss(data, query, k, num_trials=10):
    total_time = 0  # Acumulador para el tiempo total
    for _ in range(num_trials):
        start_time = time.time()  # Inicio del temporizador
        d = len(data[0])  # Dimensionality of the feature vectors
        index = faiss.IndexFlatL2(d)  # L2 distance
        index.add(np.array(data, dtype='float32'))  # Add data to the index
        D, I = index.search(np.array([query], dtype='float32'), k)  # Perform the search k+1 para no contar la query
        end_time = time.time()  # Fin del temporizador
        total_time += end_time - start_time  # Acumular el tiempo de ejecución
    average_time = total_time / num_trials  # Calcular el promedio
    return I[0], D[0], average_time

# Testbench 1k

**Extracción de características**

In [10]:
features = extract_n_features(1000, 64, 8)

# Guardar las características en un archivo CSV
output_file = 'caracteristicas_1000.txt'
with open(output_file, 'w') as f:
    for audio_path, feature_vector in features.items():
        feature_str = ','.join(map(str, feature_vector))
        f.write(feature_str + '\n')

print(f'Características guardadas en {output_file}')

# Variables para tests
datatrain = pd.read_csv('caracteristicas_1000.txt', delimiter=',',header=None)
k = 8
ind_q = 5

Características guardadas en caracteristicas_1000.txt


**Range KNN Priority queue**

In [ ]:
query = datatrain.iloc[ind_q]
#dataT = datatrain.drop([ind_q], axis=0, inplace=False)
dataT = datatrain
result = knnSearch(dataT.values, query, k)
# Promedio de tiempo
total_time = 0
for _ in range(10):
    start_time = time.time()
    result = knnSearch(dataT.values, query, k)
    end_time = time.time()
    total_time += end_time - start_time
average_time = total_time

print(f'Resultado para k={k}: {result}')
print(f'Tiempo promedio: {average_time} segundos') 

Resultado para k=8: [(5, 0.0), (785, 29.24923060155461), (979, 29.42602781609264), (640, 30.116354291957794), (762, 30.265184933752206), (542, 31.792580509416787), (107, 33.27158275278003), (73, 34.583221927244736)]
Tiempo promedio: 5.341972351074219 segundos


**Range KNN Radio**

In [ ]:
D = genDistancias(datatrain.values, 5000)

# Calcular percentiles para elegir r1, r2, y r3
percentiles = np.percentile(D, [10, 20, 30]) # tmb puede ser [25, 50, 75]
r1, r2, r3 = percentiles[0], percentiles[1], percentiles[2]

# calcular la busqueda por rango
radios_obtenidos = [r1, r2, r3]

for radio in radios_obtenidos:
    query = datatrain.iloc[ind_q]
    dataT = datatrain
    # Promedio
    total_time = 0
    for _ in range(10):
        start_time = time.time()
        result = rangeSearch(dataT.values, query, radio, ind_q)
        end_time = time.time()
        total_time += end_time - start_time
    average_time = total_time / 10

    print(f'Resultado para radio={radio}: {result}')
    print(f'Tiempo promedio: {average_time} segundos')

Resultado para radio=56.655035348566905: [5, 785, 979, 640, 762, 542, 107, 73, 761, 215, 206, 225, 312, 151, 211, 13, 169, 355, 51, 114, 219, 34, 679, 869, 190, 36, 843, 105, 80, 177, 48, 453, 12, 482, 186, 880, 438, 104, 300, 794, 115, 876, 166, 694, 62, 912, 209]
Tiempo promedio: 0.30761802196502686 segundos
Resultado para radio=71.59230317636263: [5, 785, 979, 640, 762, 542, 107, 73, 761, 215, 206, 225, 312, 151, 211, 13, 169, 355, 51, 114, 219, 34, 679, 869, 190, 36, 843, 105, 80, 177, 48, 453, 12, 482, 186, 880, 438, 104, 300, 794, 115, 876, 166, 694, 62, 912, 209, 152, 125, 15, 886, 349, 410, 85, 129, 529, 77, 158, 131, 599, 111, 747, 167, 656, 600, 573, 951, 119, 241, 75, 63, 911, 914, 815, 295, 145, 675, 958]
Tiempo promedio: 0.28712096214294436 segundos
Resultado para radio=85.68105238193611: [5, 785, 979, 640, 762, 542, 107, 73, 761, 215, 206, 225, 312, 151, 211, 13, 169, 355, 51, 114, 219, 34, 679, 869, 190, 36, 843, 105, 80, 177, 48, 453, 12, 482, 186, 880, 438, 104, 300, 7

**RTree KNN**

In [ ]:
D = len(datatrain.columns)  

# Create a new RTree index
prop = rtree.index.Property()
prop.dimension = D    # dimension del vector caracteristico
prop.buffering_capacity = 8    # Cantidad maxima de MBRs en un nodo 
ind = rtree.index.Index(properties = prop)

#Generar los datos    
data = datatrain.values
query = data[ind_q]

# insertar los puntos                
for i in range(data.shape[0]):
    ind.insert(i, data[i])    

knn_rtree(data, query, k, 64)

La consulta tomó en promedio 0.0003629446029663086 segundos
Las k:8 canciones más cercanas a la consulta son (contar la query): [5, 785, 979, 640, 762, 542, 107, 73]


**HighD**

In [ ]:
#Generar los datos    
data = datatrain.values
query = data[5]

faiss_knn_results, faiss_knn_distances, average_time = knn_faiss(data, query, k=8)

print(f"Tiempo de ejecución promedio de knn_faiss: {average_time} segundos")
print(f'Los k={k} más cercanos a la consulta son:{faiss_knn_results}') # faiss_knn_results[1:]
print(f'Las distancias a los k={k} vecinos más cercanos son:{faiss_knn_distances}') # faiss_knn_results[1:]

Tiempo de ejecución promedio de knn_faiss: 0.0007256031036376953 segundos
Los k=8 más cercanos a la consulta son:[  5 785 979 640 762 542 107  73]
Las distancias a los k=8 vecinos más cercanos son:[   0.       855.5176   865.8911   906.99475  915.9813  1010.76825
 1106.9984  1195.9993 ]


# Testbench 2k

**Extracción de características**

In [ ]:
features = extract_n_features(2000, 64, 8)

# Guardar las características en un archivo CSV
output_file = 'caracteristicas_2000.txt'
with open(output_file, 'w') as f:
    for audio_path, feature_vector in features.items():
        feature_str = ','.join(map(str, feature_vector))
        f.write(feature_str + '\n')

print(f'Características guardadas en {output_file}')

# Variables para tests
datatrain = pd.read_csv('caracteristicas_2000.txt', delimiter=',',header=None)
k = 8
ind_q = 5

[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!


Características guardadas en caracteristicas_2000.txt


**Range KNN Priority queue**

In [ ]:
query = datatrain.iloc[ind_q]
#dataT = datatrain.drop([ind_q], axis=0, inplace=False)
dataT = datatrain
result = knnSearch(dataT.values, query, k)
# Promedio de tiempo
total_time = 0
for _ in range(10):
    start_time = time.time()
    result = knnSearch(dataT.values, query, k)
    end_time = time.time()
    total_time += end_time - start_time
average_time = total_time

print(f'Resultado para k={k}: {result}')
print(f'Tiempo promedio: {average_time} segundos') 

Resultado para k=8: [(5, 0.0), (785, 29.24923060155461), (979, 29.42602781609264), (640, 30.116354291957794), (762, 30.265184933752206), (542, 31.792580509416787), (107, 33.27158275278003), (73, 34.583221927244736)]
Tiempo promedio: 5.0945885181427 segundos


**Range KNN Radio**

In [ ]:
D = genDistancias(datatrain.values, 8000)

# Calcular percentiles para elegir r1, r2, y r3
percentiles = np.percentile(D, [10, 20, 30]) # tmb puede ser [25, 50, 75]
r1, r2, r3 = percentiles[0], percentiles[1], percentiles[2]

# calcular la busqueda por rango
radios_obtenidos = [r1, r2, r3]

for radio in radios_obtenidos:
    query = datatrain.iloc[ind_q]
    dataT = datatrain
    # Promedio
    total_time = 0
    for _ in range(10):
        start_time = time.time()
        result = rangeSearch(dataT.values, query, radio, ind_q)
        end_time = time.time()
        total_time += end_time - start_time
    average_time = total_time / 10

    print(f'Resultado para radio={radio}: {result}')
    print(f'Tiempo promedio: {average_time} segundos')

Resultado para radio=57.718130435274: [5, 785, 979, 640, 762, 542, 107, 73, 761, 215, 206, 225, 312, 151, 211, 13, 169, 355, 51, 114, 219, 34, 679, 869, 190, 36, 843, 105, 80, 177, 48, 453, 12, 482, 186, 880, 438, 104, 300, 794, 115, 876, 166, 694, 62, 912, 209, 152, 125, 15]
Tiempo promedio: 0.2738438606262207 segundos
Resultado para radio=72.19710202750538: [5, 785, 979, 640, 762, 542, 107, 73, 761, 215, 206, 225, 312, 151, 211, 13, 169, 355, 51, 114, 219, 34, 679, 869, 190, 36, 843, 105, 80, 177, 48, 453, 12, 482, 186, 880, 438, 104, 300, 794, 115, 876, 166, 694, 62, 912, 209, 152, 125, 15, 886, 349, 410, 85, 129, 529, 77, 158, 131, 599, 111, 747, 167, 656, 600, 573, 951, 119, 241, 75, 63, 911, 914, 815, 295, 145, 675, 958, 765, 212]
Tiempo promedio: 0.31832988262176515 segundos
Resultado para radio=86.60494440587891: [5, 785, 979, 640, 762, 542, 107, 73, 761, 215, 206, 225, 312, 151, 211, 13, 169, 355, 51, 114, 219, 34, 679, 869, 190, 36, 843, 105, 80, 177, 48, 453, 12, 482, 186, 8

**RTree KNN**

In [ ]:
D = len(datatrain.columns)  

# Create a new RTree index
prop = rtree.index.Property()
prop.dimension = D    # dimension del vector caracteristico
prop.buffering_capacity = 8    # Cantidad maxima de MBRs en un nodo 
ind = rtree.index.Index(properties = prop)

#Generar los datos    
data = datatrain.values
query = data[ind_q]

# insertar los puntos                
for i in range(data.shape[0]):
    ind.insert(i, data[i])    

knn_rtree(data, query, k, 64)

La consulta tomó en promedio 0.000470733642578125 segundos
Las k:8 canciones más cercanas a la consulta son (contar la query): [5, 785, 979, 640, 762, 542, 107, 73]


**HighD**

In [ ]:
#Generar los datos    
data = datatrain.values
query = data[5]

faiss_knn_results, faiss_knn_distances, average_time = knn_faiss(data, query, k=8)

print(f"Tiempo de ejecución promedio de knn_faiss: {average_time} segundos")
print(f'Los k={k} más cercanos a la consulta son:{faiss_knn_results}') # faiss_knn_results[1:]
print(f'Las distancias a los k={k} vecinos más cercanos son:{faiss_knn_distances}') # faiss_knn_results[1:]

Tiempo de ejecución promedio de knn_faiss: 0.0007380008697509766 segundos
Los k=8 más cercanos a la consulta son:[  5 785 979 640 762 542 107  73]
Las distancias a los k=8 vecinos más cercanos son:[   0.       855.5176   865.8911   906.99475  915.9813  1010.76825
 1106.9984  1195.9993 ]


# Testbench 4k

**Extracción de características**

In [ ]:
features = extract_n_features(4000, 64, 8)

# Guardar las características en un archivo CSV
output_file = 'caracteristicas_4000.txt'
with open(output_file, 'w') as f:
    for audio_path, feature_vector in features.items():
        feature_str = ','.join(map(str, feature_vector))
        f.write(feature_str + '\n')

print(f'Características guardadas en {output_file}')

# Variables para tests
datatrain = pd.read_csv('caracteristicas_4000.txt', delimiter=',',header=None)
k = 8
ind_q = 5

[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1771] error: part2_3_length (3264) too large for available bit count (3224)
[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!


Características guardadas en caracteristicas_4000.txt


**Range KNN Priority queue**

In [ ]:
query = datatrain.iloc[ind_q]
#dataT = datatrain.drop([ind_q], axis=0, inplace=False)
dataT = datatrain
result = knnSearch(dataT.values, query, k)
# Promedio de tiempo
total_time = 0
for _ in range(10):
    start_time = time.time()
    result = knnSearch(dataT.values, query, k)
    end_time = time.time()
    total_time += end_time - start_time
average_time = total_time

print(f'Resultado para k={k}: {result}')
print(f'Tiempo promedio: {average_time} segundos') 

Resultado para k=8: [(5, 0.0), (31, 21.27868951349949), (9, 26.562299151551848), (200, 34.62912923822806), (2993, 36.522520758226264), (631, 46.58289851191702), (2873, 47.80771715933784), (3059, 47.908723321964956)]
Tiempo promedio: 12.864285469055176 segundos


**Range KNN Radio**

In [ ]:
D = genDistancias(datatrain.values, 12000)

# Calcular percentiles para elegir r1, r2, y r3
percentiles = np.percentile(D, [10, 20, 30]) # tmb puede ser [25, 50, 75]
r1, r2, r3 = percentiles[0], percentiles[1], percentiles[2]

# calcular la busqueda por rango
radios_obtenidos = [r1, r2, r3]

for radio in radios_obtenidos:
    query = datatrain.iloc[ind_q]
    dataT = datatrain
    # Promedio
    total_time = 0
    for _ in range(10):
        start_time = time.time()
        result = rangeSearch(dataT.values, query, radio, ind_q)
        end_time = time.time()
        total_time += end_time - start_time
    average_time = total_time / 10

    print(f'Resultado para radio={radio}: {result}')
    print(f'Tiempo promedio: {average_time} segundos')

Resultado para radio=62.631407362456756: [5, 31, 9, 200, 2993, 631, 2873, 3059, 2277, 2707, 3681, 1274, 2036, 2180, 719, 1661, 43, 2621, 1307, 22, 2194, 1430, 3652, 3637, 174, 1023, 238, 3395, 145, 162, 128, 587, 1673, 3440, 1406, 1323, 2848, 658, 1114, 1766, 2487, 1967, 372, 1250, 3193, 2023, 3197, 3150, 3123, 3054, 3137, 119, 2365, 68, 3625, 1630, 2254, 2549, 3755, 3626, 2840, 1741, 393, 14, 1066, 1791]
Tiempo promedio: 0.6736599922180175 segundos
Resultado para radio=79.35393718762391: [5, 31, 9, 200, 2993, 631, 2873, 3059, 2277, 2707, 3681, 1274, 2036, 2180, 719, 1661, 43, 2621, 1307, 22, 2194, 1430, 3652, 3637, 174, 1023, 238, 3395, 145, 162, 128, 587, 1673, 3440, 1406, 1323, 2848, 658, 1114, 1766, 2487, 1967, 372, 1250, 3193, 2023, 3197, 3150, 3123, 3054, 3137, 119, 2365, 68, 3625, 1630, 2254, 2549, 3755, 3626, 2840, 1741, 393, 14, 1066, 1791, 2287, 478, 3276, 3678, 2702, 1105, 1382, 3222, 1537, 2540, 827, 3363, 159, 2567, 3697, 523, 2109, 3664, 212, 1301, 3501, 310, 1446, 1059, 

**RTree KNN**

In [ ]:
D = len(datatrain.columns)  

# Create a new RTree index
prop = rtree.index.Property()
prop.dimension = D    # dimension del vector caracteristico
prop.buffering_capacity = 8    # Cantidad maxima de MBRs en un nodo 
ind = rtree.index.Index(properties = prop)

#Generar los datos    
data = datatrain.values
query = data[ind_q]

# insertar los puntos                
for i in range(data.shape[0]):
    ind.insert(i, data[i])    

knn_rtree(data, query, k, 64)

La consulta tomó en promedio 0.002820467948913574 segundos
Las k:8 canciones más cercanas a la consulta son (contar la query): [5, 31, 9, 200, 2993, 631, 2873, 3059]


**HighD**

In [ ]:
#Generar los datos    
data = datatrain.values
query = data[5]

faiss_knn_results, faiss_knn_distances, average_time = knn_faiss(data, query, k=8)

print(f"Tiempo de ejecución promedio de knn_faiss: {average_time} segundos")
print(f'Los k={k} más cercanos a la consulta son:{faiss_knn_results}') # faiss_knn_results[1:]
print(f'Las distancias a los k={k} vecinos más cercanos son:{faiss_knn_distances}') # faiss_knn_results[1:]

Tiempo de ejecución promedio de knn_faiss: 0.0010390996932983399 segundos
Los k=8 más cercanos a la consulta son:[   5   31    9  200 2993  631 2873 3059]
Las distancias a los k=8 vecinos más cercanos son:[   0.       452.78265  705.5557  1199.1769  1333.8945  2169.966
 2285.578   2295.246  ]


# Testbench 8k

**Extracción de características**

In [ ]:
features = extract_n_features(8000, 64, 8)

# Guardar las características en un archivo CSV
output_file = 'caracteristicas_8000.txt'
with open(output_file, 'w') as f:
    for audio_path, feature_vector in features.items():
        feature_str = ','.join(map(str, feature_vector))
        f.write(feature_str + '\n')

print(f'Características guardadas en {output_file}')

# Variables para tests
datatrain = pd.read_csv('caracteristicas_8000.txt', delimiter=',',header=None)
k = 8
ind_q = 5

[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!


KeyboardInterrupt: 

[src/libmpg123/layer3.c:INT123_do_layer3():1771] error: part2_3_length (3264) too large for available bit count (3224)


**Range KNN Priority queue**

In [ ]:
query = datatrain.iloc[ind_q]
#dataT = datatrain.drop([ind_q], axis=0, inplace=False)
dataT = datatrain
result = knnSearch(dataT.values, query, k)
# Promedio de tiempo
total_time = 0
for _ in range(10):
    start_time = time.time()
    result = knnSearch(dataT.values, query, k)
    end_time = time.time()
    total_time += end_time - start_time
average_time = total_time

print(f'Resultado para k={k}: {result}')
print(f'Tiempo promedio: {average_time} segundos') 

Resultado para k=8: [(5, 0.0), (31, 21.27868951349949), (9, 26.562299151551848), (200, 34.62912923822806), (2993, 36.522520758226264), (631, 46.58289851191702), (2873, 47.80771715933784), (3059, 47.908723321964956)]
Tiempo promedio: 12.864285469055176 segundos


**Range KNN Radio**

In [ ]:
D = genDistancias(datatrain.values, 15000)

# Calcular percentiles para elegir r1, r2, y r3
percentiles = np.percentile(D, [10, 20, 30]) # tmb puede ser [25, 50, 75]
r1, r2, r3 = percentiles[0], percentiles[1], percentiles[2]

# calcular la busqueda por rango
radios_obtenidos = [r1, r2, r3]

for radio in radios_obtenidos:
    query = datatrain.iloc[ind_q]
    dataT = datatrain
    # Promedio
    total_time = 0
    for _ in range(10):
        start_time = time.time()
        result = rangeSearch(dataT.values, query, radio, ind_q)
        end_time = time.time()
        total_time += end_time - start_time
    average_time = total_time / 10

    print(f'Resultado para radio={radio}: {result}')
    print(f'Tiempo promedio: {average_time} segundos')

Resultado para radio=62.631407362456756: [5, 31, 9, 200, 2993, 631, 2873, 3059, 2277, 2707, 3681, 1274, 2036, 2180, 719, 1661, 43, 2621, 1307, 22, 2194, 1430, 3652, 3637, 174, 1023, 238, 3395, 145, 162, 128, 587, 1673, 3440, 1406, 1323, 2848, 658, 1114, 1766, 2487, 1967, 372, 1250, 3193, 2023, 3197, 3150, 3123, 3054, 3137, 119, 2365, 68, 3625, 1630, 2254, 2549, 3755, 3626, 2840, 1741, 393, 14, 1066, 1791]
Tiempo promedio: 0.6736599922180175 segundos
Resultado para radio=79.35393718762391: [5, 31, 9, 200, 2993, 631, 2873, 3059, 2277, 2707, 3681, 1274, 2036, 2180, 719, 1661, 43, 2621, 1307, 22, 2194, 1430, 3652, 3637, 174, 1023, 238, 3395, 145, 162, 128, 587, 1673, 3440, 1406, 1323, 2848, 658, 1114, 1766, 2487, 1967, 372, 1250, 3193, 2023, 3197, 3150, 3123, 3054, 3137, 119, 2365, 68, 3625, 1630, 2254, 2549, 3755, 3626, 2840, 1741, 393, 14, 1066, 1791, 2287, 478, 3276, 3678, 2702, 1105, 1382, 3222, 1537, 2540, 827, 3363, 159, 2567, 3697, 523, 2109, 3664, 212, 1301, 3501, 310, 1446, 1059, 

**RTree KNN**

In [ ]:
D = len(datatrain.columns)  

# Create a new RTree index
prop = rtree.index.Property()
prop.dimension = D    # dimension del vector caracteristico
prop.buffering_capacity = 8    # Cantidad maxima de MBRs en un nodo 
ind = rtree.index.Index(properties = prop)

#Generar los datos    
data = datatrain.values
query = data[ind_q]

# insertar los puntos                
for i in range(data.shape[0]):
    ind.insert(i, data[i])    

knn_rtree(data, query, k, 64)

La consulta tomó en promedio 0.002820467948913574 segundos
Las k:8 canciones más cercanas a la consulta son (contar la query): [5, 31, 9, 200, 2993, 631, 2873, 3059]


**HighD**

In [ ]:
#Generar los datos    
data = datatrain.values
query = data[5]

faiss_knn_results, faiss_knn_distances, average_time = knn_faiss(data, query, k=8)

print(f"Tiempo de ejecución promedio de knn_faiss: {average_time} segundos")
print(f'Los k={k} más cercanos a la consulta son:{faiss_knn_results}') # faiss_knn_results[1:]
print(f'Las distancias a los k={k} vecinos más cercanos son:{faiss_knn_distances}') # faiss_knn_results[1:]

Tiempo de ejecución promedio de knn_faiss: 0.0010390996932983399 segundos
Los k=8 más cercanos a la consulta son:[   5   31    9  200 2993  631 2873 3059]
Las distancias a los k=8 vecinos más cercanos son:[   0.       452.78265  705.5557  1199.1769  1333.8945  2169.966
 2285.578   2295.246  ]


# Testbench 16k

**Extracción de características**

In [ ]:
features = extract_n_features(16000, 64, 8)

# Guardar las características en un archivo CSV
output_file = 'caracteristicas_16000.txt'
with open(output_file, 'w') as f:
    for audio_path, feature_vector in features.items():
        feature_str = ','.join(map(str, feature_vector))
        f.write(feature_str + '\n')

print(f'Características guardadas en {output_file}')

# Variables para tests
datatrain = pd.read_csv('caracteristicas_16000.txt', delimiter=',',header=None)
k = 8
ind_q = 5

[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1771] error: part2_3_length (3264) too large for available bit count (3224)
[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!


Características guardadas en caracteristicas_4000.txt


**Range KNN Priority queue**

In [ ]:
query = datatrain.iloc[ind_q]
#dataT = datatrain.drop([ind_q], axis=0, inplace=False)
dataT = datatrain
result = knnSearch(dataT.values, query, k)
# Promedio de tiempo
total_time = 0
for _ in range(10):
    start_time = time.time()
    result = knnSearch(dataT.values, query, k)
    end_time = time.time()
    total_time += end_time - start_time
average_time = total_time

print(f'Resultado para k={k}: {result}')
print(f'Tiempo promedio: {average_time} segundos') 

Resultado para k=8: [(5, 0.0), (31, 21.27868951349949), (9, 26.562299151551848), (200, 34.62912923822806), (2993, 36.522520758226264), (631, 46.58289851191702), (2873, 47.80771715933784), (3059, 47.908723321964956)]
Tiempo promedio: 12.864285469055176 segundos


**Range KNN Radio**

In [ ]:
D = genDistancias(datatrain.values, 24000)

# Calcular percentiles para elegir r1, r2, y r3
percentiles = np.percentile(D, [10, 20, 30]) # tmb puede ser [25, 50, 75]
r1, r2, r3 = percentiles[0], percentiles[1], percentiles[2]

# calcular la busqueda por rango
radios_obtenidos = [r1, r2, r3]

for radio in radios_obtenidos:
    query = datatrain.iloc[ind_q]
    dataT = datatrain
    # Promedio
    total_time = 0
    for _ in range(10):
        start_time = time.time()
        result = rangeSearch(dataT.values, query, radio, ind_q)
        end_time = time.time()
        total_time += end_time - start_time
    average_time = total_time / 10

    print(f'Resultado para radio={radio}: {result}')
    print(f'Tiempo promedio: {average_time} segundos')

Resultado para radio=62.631407362456756: [5, 31, 9, 200, 2993, 631, 2873, 3059, 2277, 2707, 3681, 1274, 2036, 2180, 719, 1661, 43, 2621, 1307, 22, 2194, 1430, 3652, 3637, 174, 1023, 238, 3395, 145, 162, 128, 587, 1673, 3440, 1406, 1323, 2848, 658, 1114, 1766, 2487, 1967, 372, 1250, 3193, 2023, 3197, 3150, 3123, 3054, 3137, 119, 2365, 68, 3625, 1630, 2254, 2549, 3755, 3626, 2840, 1741, 393, 14, 1066, 1791]
Tiempo promedio: 0.6736599922180175 segundos
Resultado para radio=79.35393718762391: [5, 31, 9, 200, 2993, 631, 2873, 3059, 2277, 2707, 3681, 1274, 2036, 2180, 719, 1661, 43, 2621, 1307, 22, 2194, 1430, 3652, 3637, 174, 1023, 238, 3395, 145, 162, 128, 587, 1673, 3440, 1406, 1323, 2848, 658, 1114, 1766, 2487, 1967, 372, 1250, 3193, 2023, 3197, 3150, 3123, 3054, 3137, 119, 2365, 68, 3625, 1630, 2254, 2549, 3755, 3626, 2840, 1741, 393, 14, 1066, 1791, 2287, 478, 3276, 3678, 2702, 1105, 1382, 3222, 1537, 2540, 827, 3363, 159, 2567, 3697, 523, 2109, 3664, 212, 1301, 3501, 310, 1446, 1059, 

**RTree KNN**

In [ ]:
D = len(datatrain.columns)  

# Create a new RTree index
prop = rtree.index.Property()
prop.dimension = D    # dimension del vector caracteristico
prop.buffering_capacity = 8    # Cantidad maxima de MBRs en un nodo 
ind = rtree.index.Index(properties = prop)

#Generar los datos    
data = datatrain.values
query = data[ind_q]

# insertar los puntos                
for i in range(data.shape[0]):
    ind.insert(i, data[i])    

knn_rtree(data, query, k, 64)

La consulta tomó en promedio 0.002820467948913574 segundos
Las k:8 canciones más cercanas a la consulta son (contar la query): [5, 31, 9, 200, 2993, 631, 2873, 3059]


**HighD**

In [ ]:
#Generar los datos    
data = datatrain.values
query = data[5]

faiss_knn_results, faiss_knn_distances, average_time = knn_faiss(data, query, k=8)

print(f"Tiempo de ejecución promedio de knn_faiss: {average_time} segundos")
print(f'Los k={k} más cercanos a la consulta son:{faiss_knn_results}') # faiss_knn_results[1:]
print(f'Las distancias a los k={k} vecinos más cercanos son:{faiss_knn_distances}') # faiss_knn_results[1:]

Tiempo de ejecución promedio de knn_faiss: 0.0010390996932983399 segundos
Los k=8 más cercanos a la consulta son:[   5   31    9  200 2993  631 2873 3059]
Las distancias a los k=8 vecinos más cercanos son:[   0.       452.78265  705.5557  1199.1769  1333.8945  2169.966
 2285.578   2295.246  ]


# Testbench 24k

**Extracción de características**

In [ ]:
features = extract_n_features(24000, 64, 8)

# Guardar las características en un archivo CSV
output_file = 'caracteristicas_24000.txt'
with open(output_file, 'w') as f:
    for audio_path, feature_vector in features.items():
        feature_str = ','.join(map(str, feature_vector))
        f.write(feature_str + '\n')

print(f'Características guardadas en {output_file}')

# Variables para tests
datatrain = pd.read_csv('caracteristicas_24000.txt', delimiter=',',header=None)
k = 8
ind_q = 5

[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1771] error: part2_3_length (3264) too large for available bit count (3224)
[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!


Características guardadas en caracteristicas_4000.txt


**Range KNN Priority queue**

In [ ]:
query = datatrain.iloc[ind_q]
#dataT = datatrain.drop([ind_q], axis=0, inplace=False)
dataT = datatrain
result = knnSearch(dataT.values, query, k)
# Promedio de tiempo
total_time = 0
for _ in range(10):
    start_time = time.time()
    result = knnSearch(dataT.values, query, k)
    end_time = time.time()
    total_time += end_time - start_time
average_time = total_time

print(f'Resultado para k={k}: {result}')
print(f'Tiempo promedio: {average_time} segundos') 

Resultado para k=8: [(5, 0.0), (31, 21.27868951349949), (9, 26.562299151551848), (200, 34.62912923822806), (2993, 36.522520758226264), (631, 46.58289851191702), (2873, 47.80771715933784), (3059, 47.908723321964956)]
Tiempo promedio: 12.864285469055176 segundos


**Range KNN Radio**

In [ ]:
D = genDistancias(datatrain.values, 30000)

# Calcular percentiles para elegir r1, r2, y r3
percentiles = np.percentile(D, [10, 20, 30]) # tmb puede ser [25, 50, 75]
r1, r2, r3 = percentiles[0], percentiles[1], percentiles[2]

# calcular la busqueda por rango
radios_obtenidos = [r1, r2, r3]

for radio in radios_obtenidos:
    query = datatrain.iloc[ind_q]
    dataT = datatrain
    # Promedio
    total_time = 0
    for _ in range(10):
        start_time = time.time()
        result = rangeSearch(dataT.values, query, radio, ind_q)
        end_time = time.time()
        total_time += end_time - start_time
    average_time = total_time / 10

    print(f'Resultado para radio={radio}: {result}')
    print(f'Tiempo promedio: {average_time} segundos')

Resultado para radio=62.631407362456756: [5, 31, 9, 200, 2993, 631, 2873, 3059, 2277, 2707, 3681, 1274, 2036, 2180, 719, 1661, 43, 2621, 1307, 22, 2194, 1430, 3652, 3637, 174, 1023, 238, 3395, 145, 162, 128, 587, 1673, 3440, 1406, 1323, 2848, 658, 1114, 1766, 2487, 1967, 372, 1250, 3193, 2023, 3197, 3150, 3123, 3054, 3137, 119, 2365, 68, 3625, 1630, 2254, 2549, 3755, 3626, 2840, 1741, 393, 14, 1066, 1791]
Tiempo promedio: 0.6736599922180175 segundos
Resultado para radio=79.35393718762391: [5, 31, 9, 200, 2993, 631, 2873, 3059, 2277, 2707, 3681, 1274, 2036, 2180, 719, 1661, 43, 2621, 1307, 22, 2194, 1430, 3652, 3637, 174, 1023, 238, 3395, 145, 162, 128, 587, 1673, 3440, 1406, 1323, 2848, 658, 1114, 1766, 2487, 1967, 372, 1250, 3193, 2023, 3197, 3150, 3123, 3054, 3137, 119, 2365, 68, 3625, 1630, 2254, 2549, 3755, 3626, 2840, 1741, 393, 14, 1066, 1791, 2287, 478, 3276, 3678, 2702, 1105, 1382, 3222, 1537, 2540, 827, 3363, 159, 2567, 3697, 523, 2109, 3664, 212, 1301, 3501, 310, 1446, 1059, 

**RTree KNN**

In [ ]:
D = len(datatrain.columns)  

# Create a new RTree index
prop = rtree.index.Property()
prop.dimension = D    # dimension del vector caracteristico
prop.buffering_capacity = 8    # Cantidad maxima de MBRs en un nodo 
ind = rtree.index.Index(properties = prop)

#Generar los datos    
data = datatrain.values
query = data[ind_q]

# insertar los puntos                
for i in range(data.shape[0]):
    ind.insert(i, data[i])    

knn_rtree(data, query, k, 64)

La consulta tomó en promedio 0.002820467948913574 segundos
Las k:8 canciones más cercanas a la consulta son (contar la query): [5, 31, 9, 200, 2993, 631, 2873, 3059]


**HighD**

In [ ]:
#Generar los datos    
data = datatrain.values
query = data[5]

faiss_knn_results, faiss_knn_distances, average_time = knn_faiss(data, query, k=8)

print(f"Tiempo de ejecución promedio de knn_faiss: {average_time} segundos")
print(f'Los k={k} más cercanos a la consulta son:{faiss_knn_results}') # faiss_knn_results[1:]
print(f'Las distancias a los k={k} vecinos más cercanos son:{faiss_knn_distances}') # faiss_knn_results[1:]

Tiempo de ejecución promedio de knn_faiss: 0.0010390996932983399 segundos
Los k=8 más cercanos a la consulta son:[   5   31    9  200 2993  631 2873 3059]
Las distancias a los k=8 vecinos más cercanos son:[   0.       452.78265  705.5557  1199.1769  1333.8945  2169.966
 2285.578   2295.246  ]
